In [ ]:
import json
import os
import pandas as pd
import random
import shutil
import zipfile
import logging
import sys
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from pathlib import Path
from PIL import Image

# Thiết lập logging với xử lý mã hóa
class EncodingSafeStreamHandler(logging.StreamHandler):
    def emit(self, record):
        try:
            msg = self.format(record)
            stream = self.stream
            stream.write(msg + self.terminator)
            self.flush()
        except UnicodeEncodeError:
            try:
                # Cố gắng mã hóa lại với ASCII, thay thế ký tự không hỗ trợ
                stream.write(msg.encode('ascii', 'replace').decode('ascii') + self.terminator)
                self.flush()
            except Exception:
                self.handleError(record)
        except Exception:
            self.handleError(record)

# Thiết lập logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        EncodingSafeStreamHandler(sys.stdout),
        logging.FileHandler('fashion_data_processing.log', encoding='utf-8')
    ]
)
logger = logging.getLogger(__name__)

class FashionDataProcessor:
    """Lop xu ly va chuan bi du lieu thoi trang."""
    
    def __init__(self, 
                 images_dir="images", 
                 captions_file="captions.json",
                 output_dir="output",
                 output_images_dir="fashion_images", 
                 min_resolution=(256, 256),
                 min_caption_length=10,
                 num_images=10000):
        """
        Khởi tạo processor với các thông số cấu hình.
        
        Args:
            images_dir (str): Thư mục chứa ảnh nguồn
            captions_file (str): File JSON chứa thông tin mô tả ảnh
            output_dir (str): Thư mục đầu ra chính
            output_images_dir (str): Thư mục chứa ảnh đã chọn
            min_resolution (tuple): Độ phân giải tối thiểu (width, height)
            min_caption_length (int): Độ dài tối thiểu của mô tả
            num_images (int): Số lượng ảnh cần chọn
        """
        self.images_dir = Path(images_dir)
        self.captions_file = Path(captions_file)
        self.output_dir = Path(output_dir)
        self.output_images_dir_name = output_images_dir
        self.output_images_dir = self.output_dir / output_images_dir
        self.output_csv = self.output_dir / "fashion_data.csv"
        self.output_zip = self.output_dir / "fashion_images.zip"
        self.min_resolution = min_resolution
        self.min_caption_length = min_caption_length
        self.num_images = num_images
        self.valid_extensions = {".jpg", ".jpeg", ".png"}
        
    def is_valid_image(self, image_path):
        """
        Kiểm tra xem ảnh có hợp lệ hay không.
        
        Args:
            image_path (Path/str): Đường dẫn đến tệp ảnh
            
        Returns:
            bool: True nếu ảnh hợp lệ, False nếu không
        """
        image_path = Path(image_path)
        
        # Kiểm tra tệp tồn tại
        if not image_path.exists():
            return False
            
        # Kiểm tra định dạng hợp lệ
        if image_path.suffix.lower() not in self.valid_extensions:
            return False
            
        try:
            with Image.open(image_path) as img:
                img = img.convert("RGB")
                width, height = img.size
                min_width, min_height = self.min_resolution
                
                # Kiểm tra độ phân giải và kênh màu
                if width < min_width or height < min_height:
                    return False
                    
                if len(img.getbands()) != 3:
                    return False
                    
                return True
        except Exception:
            return False
    
    def load_captions(self):
        """Đọc và trả về dữ liệu caption từ file JSON."""
        try:
            with open(self.captions_file, "r", encoding="utf-8") as f:
                return json.load(f)
        except (json.JSONDecodeError, FileNotFoundError) as e:
            logger.error(f"Lỗi khi đọc file caption: {e}")
            raise
    
    def filter_valid_images(self, captions, max_workers=None):
        """
        Lọc các ảnh hợp lệ có trong thư mục và có caption
        
        Args:
            captions (dict): Dictionary chứa dữ liệu caption
            max_workers: Số lượng worker tối đa cho xử lý song song
            
        Returns:
            list: Danh sách tên file ảnh hợp lệ
        """
        valid_images = []
        total_images = len(captions)
        
        logger.info(f"Dang kiem tra {total_images} anh...")
        
        # Dùng ThreadPoolExecutor để kiểm tra song song các ảnh
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            check_image = partial(self._check_image_validity, captions=captions)
            results = list(executor.map(check_image, list(captions.keys())))
            valid_images = [img for img, valid in zip(captions.keys(), results) if valid]
            
        return valid_images
    
    def _check_image_validity(self, img_name, captions):
        """Kiểm tra tính hợp lệ của một ảnh."""
        img_path = self.images_dir / img_name
        return self.is_valid_image(img_path) and img_name in captions
        
    def select_random_images(self, valid_images):
        """
        Chọn ngẫu nhiên số lượng ảnh yêu cầu từ danh sách ảnh hợp lệ
        
        Args:
            valid_images (list): Danh sách tên file ảnh hợp lệ
            
        Returns:
            list: Danh sách tên file ảnh đã chọn
        """
        if len(valid_images) < self.num_images:
            logger.error(f"Khong du anh hop le: co {len(valid_images)}, can {self.num_images}")
            raise ValueError(f"Khong du anh hop le: co {len(valid_images)}, can {self.num_images}")
        
        return random.sample(valid_images, self.num_images)
    
    def create_dataframe(self, selected_images, captions):
        """
        Tạo DataFrame và lọc caption
        
        Args:
            selected_images (list): Danh sách tên file ảnh đã chọn
            captions (dict): Dictionary chứa dữ liệu caption
            
        Returns:
            DataFrame: DataFrame đã lọc
        """
        data = {
            "image_name": selected_images,
            "image_path": [f"{self.output_images_dir_name}/{img_name}" for img_name in selected_images],
            "caption": [captions[img_name] for img_name in selected_images]
        }
        
        df = pd.DataFrame(data)
        
        # Lọc caption dài hơn min_caption_length ký tự
        initial_count = len(df)
        df = df[df["caption"].str.len() > self.min_caption_length]
        logger.info(f"Sau khi loc caption co do dai > {self.min_caption_length}: {len(df)} muc (da loai bo {initial_count - len(df)})")
        
        return df
    
    def copy_selected_images(self, df, max_workers=None):
        """
        Sao chép các ảnh đã chọn sang thư mục đầu ra
        
        Args:
            df (DataFrame): DataFrame chứa thông tin ảnh đã lọc
            max_workers: Số lượng worker tối đa cho xử lý song song
            
        Returns:
            int: Số lượng ảnh đã sao chép
        """
        # Tạo thư mục đầu ra nếu chưa tồn tại
        self.output_images_dir.mkdir(parents=True, exist_ok=True)
        
        # Lấy danh sách tên file sau khi lọc caption
        selected_images = df["image_name"].tolist()
        
        count = 0
        copy_tasks = []
        
        # Chuẩn bị danh sách công việc sao chép
        for img_name in selected_images:
            src_path = self.images_dir / img_name
            dst_path = self.output_images_dir / img_name
            copy_tasks.append((src_path, dst_path))
        
        # Dùng ThreadPoolExecutor để sao chép song song
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            results = list(executor.map(self._copy_image, copy_tasks))
            count = sum(1 for success in results if success)
            
        return count
    
    def _copy_image(self, paths):
        """Sao chép một ảnh từ nguồn đến đích."""
        src_path, dst_path = paths
        try:
            if self.is_valid_image(src_path):
                shutil.copy2(src_path, dst_path)
                return True
            return False
        except Exception as e:
            logger.error(f"Lỗi khi sao chép ảnh {src_path}: {e}")
            return False
            
    def create_zip(self):
        """Tao file ZIP chua thu muc anh."""
        try:
            with zipfile.ZipFile(self.output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for root, _, files in os.walk(self.output_images_dir):
                    for file in files:
                        file_path = Path(root) / file
                        arcname = file_path.relative_to(self.output_dir)
                        zipf.write(file_path, arcname)
            
            logger.info(f"Da tao file ZIP: {self.output_zip}")
            return True
        except Exception as e:
            logger.error(f"Loi khi tao file ZIP: {str(e)}")
            return False
    
    def process(self):
        """Thuc hien toan bo quy trinh xu ly."""
        try:
            # Bước 1: Đọc dữ liệu caption
            logger.info("Dang doc file caption...")
            captions = self.load_captions()
            logger.info(f"Da doc {len(captions)} caption tu file.")
            
            # Bước 2: Lọc ảnh hợp lệ
            valid_images = self.filter_valid_images(captions)
            logger.info(f"Tim thay {len(valid_images)} anh hop le co caption.")
            
            # Bước 3: Chọn ngẫu nhiên ảnh
            selected_images = self.select_random_images(valid_images)
            logger.info(f"Da chon {len(selected_images)} anh de xu ly.")
            
            # Bước 4: Tạo DataFrame và lọc caption
            df = self.create_dataframe(selected_images, captions)
            
            # Tạo thư mục đầu ra nếu chưa tồn tại
            self.output_dir.mkdir(parents=True, exist_ok=True)
            
            # Lưu DataFrame
            df.to_csv(self.output_csv, index=False)
            logger.info(f"Da luu {self.output_csv} voi {len(df)} muc.")
            
            # Bước 5: Sao chép ảnh đã chọn
            copied_count = self.copy_selected_images(df)
            logger.info(f"Da sao chep {copied_count} anh vao {self.output_images_dir}.")
            
            # Bước 6: Tạo file ZIP
            self.create_zip()
            
            logger.info("Hoan tat xu ly du lieu thoi trang!")
            return True
            
        except Exception as e:
            logger.error(f"Loi trong qua trinh xu ly: {str(e)}")
            return False


if __name__ == "__main__":
    # Định nghĩa tham số (có thể chuyển sang sử dụng argparse trong tương lai)
    processor = FashionDataProcessor(
        images_dir="images",
        captions_file="captions.json",
        output_dir="fashion_images_10000",
        output_images_dir="fashion_images_10000",
        min_resolution=(256, 256),
        min_caption_length=10,
        num_images=10000
    )
    
    # Thực hiện quy trình xử lý
    processor.process()


2025-05-25 11:22:02,892 - INFO - Dang doc file caption...
2025-05-25 11:22:03,011 - INFO - Da doc 42544 caption tu file.
2025-05-25 11:22:03,014 - INFO - Dang kiem tra 42544 anh...
2025-05-25 11:24:14,165 - INFO - Tim thay 42544 anh hop le co caption.
2025-05-25 11:24:14,165 - INFO - Da chon 10000 anh de xu ly.
2025-05-25 11:24:14,165 - INFO - Sau khi loc caption co do dai > 10: 9998 muc (da loai bo 2)
2025-05-25 11:24:14,181 - INFO - Da luu fashion_images_10000ashion_data.csv voi 100 muc.
2025-05-25 11:24:14,588 - INFO - Da sao chep 10000 anh vao fashion_images_10000ashion_images_10000.
2025-05-25 11:24:15,542 - INFO - Da tao file ZIP: fashion_images_10000ashion_images.zip
2025-05-25 11:24:15,543 - INFO - Hoan tat xu ly du lieu thoi trang!
      
